In [1]:
##training
import os,pickle
import pandas as pd
from utils.DataProcess import Start
from utils.ClooingTowerOptimize import CT_opt

opt = CT_opt()
opt.df = Start('HL_8')

opt.train()
with open(os.path.join('model','regr.pkl'), 'wb') as f:
    pickle.dump(opt, f)

冷卻水塔 MAE: 18.6
冰水主機 MAE: 23.99
冷卻水泵 MAE: 2.75


In [66]:
# eval plot
import os,pickle
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
from utils.DataProcess import Start
from utils.ClooingTowerOptimize import CT_opt
with open(os.path.join('model','regr.pkl'), 'rb') as p:
    opt = pickle.load(p)


vs = []
for c in tqdm(range(2000)):
    try:
        opt.predict(c)
        v,fig = opt.plot(21,29)
        vs.append(v)
    except:
        pass


fig = go.Figure()
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=opt.df.condenser_supply_temp[-1000:][::-1],mode='lines',name='Manual'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=np.array(vs),mode='lines',name='AI'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=opt.df.Wet_bulb_temp[-1000:][::-1],mode='lines',name='Wet bulb'))

100%|██████████| 2000/2000 [00:36<00:00, 54.44it/s]


In [83]:
from plotly.graph_objs.layout import YAxis,XAxis,Margin
layout = go.Layout(
    # title="{}".format(self.df_last.Datetime),
    yaxis=YAxis(
        title="<b> Approach </b>"
    ),
    yaxis2 = YAxis(
        title="<b> Wet bulb temp </b>",
        overlaying= 'y', 
        side= 'right',
    ),
    yaxis3 = YAxis(
        title="<b> Wet bulb temp </b>",
        overlaying= 'y', 
        side= 'right',
    ),
    xaxis=dict(
        title="<b> Cost (Kw)</b>"
    ),
)
fig =  go.Figure(layout=layout)
fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=np.array(vs),name='AI',yaxis='y2',opacity=0.5))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=opt.df.Wet_bulb_temp[-2000:][::-1],name='Wet bulb',yaxis='y2'))
# fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=opt.df.CT_eff[-2000:][::-1],name='CT_eff',yaxis='y1'))
# fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=opt.df.system_KPI[-2000:][::-1],name='KPI',yaxis='y3'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=opt.df.condenser_supply_temp[-2000:][::-1],name='Manual',yaxis='y2',opacity=0.5))




In [73]:
fig =  go.Figure(layout=layout)
fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=np.array(vs-opt.df.system_KPI[-2000:][::-1]),name='KPI',yaxis='y1',opacity=0.5))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=opt.df.Wet_bulb_temp[-2000:][::-1],name='Wet bulb',yaxis='y2'))

In [64]:
# eval plot
import os,pickle
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
from utils.DataProcess import Start
from utils.ClooingTowerOptimize import CT_opt
with open(os.path.join('model','regr.pkl'), 'rb') as p:
    opt = pickle.load(p)


vs = []
for c in tqdm(range(1000)):
    try:
        opt.predict(c)
        v,fig = opt.plot(21,29)
        vs.append(v)
    except:
        pass


fig = go.Figure()
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=opt.df.condenser_supply_temp[-1000:][::-1],mode='lines',name='Manual'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=np.array(vs),mode='lines',name='AI'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=opt.df.Enthalpy[-1000:][::-1],mode='lines',name='Wet bulb'))

100%|██████████| 1000/1000 [00:18<00:00, 53.05it/s]


In [3]:
a = opt.predict(1)
a,b = opt.plot(21,29)

In [4]:
a,b = opt.plot(21,29)

In [5]:
opt.df.plot.scatter(y='CT_VFD_OUT',x='CT_eff')

In [6]:
b

In [ ]:
df.Enthalpy

In [81]:
import plotly.express as px
df = opt.df.copy()
df = df[df.Enthalpy>0]
px.scatter(
    data_frame=df,
    x='CT_eff',
    y='CT_Total_KW',
    color='Enthalpy'
)

In [28]:
import plotly.express as px
df = opt.df.copy()
df = df[df.Wet_bulb_temp>0]
px.scatter(
    data_frame=df,
    x='Approach',
    y='CT_Total_KW',
    color='Wet_bulb_temp'
)